In [8]:
!pip install lxml
!pip install argparse
!pip install bs4
!pip install unicodecsv

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Using cached unicodecsv-0.14.1-py3-none-any.whl


In [31]:
#!/usr/bin/env python
from datetime import datetime
from time import time
from lxml import html,etree
from reviews_final import scrape, write_in_csv
import pandas as pd
import requests,re
import os,sys
import unicodecsv as csv
import argparse

def parse(locality,checkin_date,checkout_date,sort):
    checkIn = checkin_date.strftime("%Y/%m/%d")
    checkOut = checkout_date.strftime("%Y/%m/%d")
    print ("Scraper Inititated for Locality:%s"%locality)
    header = {

                            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
            }
    # TA rendering the autocomplete list using this API
    print ("Finding search result page URL")
    geo_url = 'https://www.tripadvisor.com/TypeAheadJson?action=API&startTime='+str(int(time()))+'&uiOrigin=GEOSCOPE&source=GEOSCOPE&interleaved=true&types=geo,theme_park&neighborhood_geos=true&link_type=hotel&details=true&max=12&injectNeighborhoods=true&query='+locality
    api_response  = requests.get(geo_url,headers=header, timeout=120).json()
    #getting the TA url for th equery from the autocomplete response
    url_from_autocomplete = "http://www.tripadvisor.com"+api_response['results'][0]['url']
    print ('URL found %s'%url_from_autocomplete)
    geo = api_response['results'][0]['value']
    #Formating date for writing to file
    a=url_from_autocomplete
    b=a.split("-")
    s="-"
    c=s.join([b[0],b[1],"oa30",b[2],b[3]])
    d=s.join([b[0],b[1],"oa60",b[2],b[3]])
    e=s.join([b[0],b[1],"oa90",b[2],b[3]])
    urllist = [a,c,d,e]

    date = checkin_date.strftime("%Y_%m_%d")+"_"+checkout_date.strftime("%Y_%m_%d")
    #form data to get the hotels list from TA for the selected date
    form_data = {'changeSet': 'TRAVEL_INFO',
            'showSnippets': 'false',
            'staydates':date,
            'uguests': '2',
            'sortOrder':sort
        
    }
    
    
    all_hotel=[]
    
    for url_from_autocomplete in urllist:
        print(url_from_autocomplete)

        headers = {
                                'Accept': 'text/javascript, text/html, application/xml, text/xml, */*',
                                'Accept-Encoding': 'gzip,deflate',
                                'Accept-Language': 'en-US,en;q=0.5',
                                'Cache-Control': 'no-cache',
                                'Connection': 'keep-alive',
                                'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
                                'Host': 'www.tripadvisor.com',
                                'Pragma': 'no-cache',
                                'Referer': url_from_autocomplete,
                                'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:28.0) Gecko/20100101 Firefox/28.0',
                                'X-Requested-With': 'XMLHttpRequest'
                            }
        cookies=  {"SetCurrency":"USD"}
        print ("Downloading search results page")
        page_response  = requests.post(url = url_from_autocomplete,data=form_data,headers = headers, cookies = cookies, verify=False)
        print ("Parsing results ")
        parser = html.fromstring(page_response.text)
        hotel_lists = parser.xpath('//div[contains(@class,"listItem")]//div[contains(@class,"listing collapsed")]')
        hotel_data = []
        if not hotel_lists:
            hotel_lists = parser.xpath('//div[contains(@class,"listItem")]//div[@class="listing "]')

        for hotel in hotel_lists:
            XPATH_HOTEL_LINK = './/a[contains(@class,"property_title")]/@href'
            XPATH_REVIEWS  = './/a[@class="review_count"]//text()'
            XPATH_RANK = './/div[@class="popindex"]//text()'
            XPATH_RATING = './/span[contains(@class,"ui_bubble_rating bubble_45")]/@alt' #update this code to get rating
            XPATH_RATING_2 = './/a[contains(@class,"ui_bubble_rating bubble_45")]/@alt' #update this code to get rating
            XPATH_HOTEL_NAME = './/a[contains(@class,"property_title")]//text()'
            XPATH_HOTEL_FEATURES = './/div[contains(@casls,"common_hotel_icons_list")]//li//text()'
            XPATH_HOTEL_PRICE = './/div[contains(@data-sizegroup,"mini-meta-price")]/text()'
            XPATH_VIEW_DEALS = './/div[contains(@data-ajax-preserve,"viewDeals")]//text()'
            XPATH_BOOKING_PROVIDER = './/div[contains(@data-sizegroup,"mini-meta-provider")]//text()'  #<span class="dekGp Ci _R S4 H3 MD">#74 of 319 hotels in Lisbon</span><span class="dekGp Ci _R S4 H3 MD">#6 of 319 hotels in Lisbon</span>
            XPATH_RATING_ORDER = './/span[contains(@class,"dekGp Ci _R S4 H3 MD")]//text()'


            raw_booking_provider = hotel.xpath(XPATH_BOOKING_PROVIDER)
            raw_no_of_deals =  hotel.xpath(XPATH_VIEW_DEALS)
            raw_hotel_link = hotel.xpath(XPATH_HOTEL_LINK)
            raw_no_of_reviews = hotel.xpath(XPATH_REVIEWS)
            raw_rank = hotel.xpath(XPATH_RANK)
            raw_rating = hotel.xpath(XPATH_RATING_2)
            raw_hotel_name = hotel.xpath(XPATH_HOTEL_NAME)
            raw_hotel_features = hotel.xpath(XPATH_HOTEL_FEATURES)
            raw_hotel_price_per_night  = hotel.xpath(XPATH_HOTEL_PRICE)
            raw_rank_order = hotel.xpath(XPATH_RATING_ORDER)

            url = 'http://www.tripadvisor.com'+raw_hotel_link[0] if raw_hotel_link else  None
            reviews = ''.join(raw_no_of_reviews).replace("reviews","").replace(",","") if raw_no_of_reviews else 0
            rank = ''.join(raw_rank) if raw_rank else None
            rating = ''.join(raw_rating).replace('of 5 bubbles','').strip() if raw_rating else None
            name = ''.join(raw_hotel_name).strip() if raw_hotel_name else None
            hotel_features = ','.join(raw_hotel_features)
            #price_per_night = ''.join(raw_hotel_price_per_night).encode('utf-8').replace('\n','') if raw_hotel_price_per_night else None
            price_per_night = ''.join(raw_hotel_price_per_night).replace('\n','') if raw_hotel_price_per_night else None
            rank_order = ''.join(raw_rank_order) if raw_rank_order else None
            no_of_deals = re.findall("all\s+?(\d+)\s+?",''.join(raw_no_of_deals))
            booking_provider = ''.join(raw_booking_provider).strip() if raw_booking_provider else None

            if no_of_deals:
                no_of_deals = no_of_deals[0]
            else:
                no_of_deals = 0

            data = {
                        'hotel_name':name,
                        'url':url,
                        'locality':locality,
                        'reviews':reviews,
                        'rank':rank,
                        'tripadvisor_rating':rating,
                        'checkOut':checkOut,
                        'checkIn':checkIn,
                        'hotel_features':hotel_features,
                        'price_per_night':price_per_night,
                        'no_of_deals':no_of_deals,
                        'booking_provider':booking_provider,
                        'raw_rank': rank_order

            }
            hotel_data.append(data)
            all_hotel.append(data)
    #Referrer is necessary to get the correct response from TA if not provided they will redirect to home page
    my_df=pd.DataFrame(all_hotel)
    print(my_df['hotel_name'])
    return all_hotel, urllist,urllist
   


In [22]:
test="2021/03/07,2021/03/10,recommended,Lisbon"

In [23]:
parser = argparse.ArgumentParser()
parser.add_argument('checkin_date',help = 'Hotel Check In Date (Format: YYYY/MM/DD')
sortorder_help = """
    available sort orders are :\n
    priceLow - hotels with lowest price,
    distLow : Hotels located near to the search center,
    recommended: highest rated hotels based on traveler reviews,
    popularity :Most popular hotels as chosen by Tipadvisor users
    """
parser.add_argument('checkout_date',help = 'Hotel Chek Out Date (Format: YYYY/MM/DD)')
parser.add_argument('sort',help = sortorder_help,default ='popularity ')
parser.add_argument('locality',help = 'Search Locality')
print(test)
args = parser.parse_args(test.split(','))
print (args)
locality = args.locality
checkin_date = datetime.strptime(args.checkin_date,"%Y/%m/%d")
checkout_date = datetime.strptime(args.checkout_date,"%Y/%m/%d")
sort= args.sort
checkIn = checkin_date.strftime("%Y/%m/%d")
checkOut = checkout_date.strftime("%Y/%m/%d")
today = datetime.now()

2021/03/07,2021/03/10,recommended,Lisbon
Namespace(checkin_date='2021/03/07', checkout_date='2021/03/10', sort='recommended', locality='Lisbon')


In [32]:
data,location_list,hlist = parse(locality,checkin_date,checkout_date,sort)

Scraper Inititated for Locality:Lisbon
Finding search result page URL
URL found http://www.tripadvisor.com/Hotels-g189158-Lisbon_Lisbon_District_Central_Portugal-Hotels.html
http://www.tripadvisor.com/Hotels-g189158-Lisbon_Lisbon_District_Central_Portugal-Hotels.html


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tripadvisor.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsing results 
http://www.tripadvisor.com/Hotels-g189158-oa30-Lisbon_Lisbon_District_Central_Portugal-Hotels.html


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tripadvisor.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsing results 
http://www.tripadvisor.com/Hotels-g189158-oa60-Lisbon_Lisbon_District_Central_Portugal-Hotels.html


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tripadvisor.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsing results 
http://www.tripadvisor.com/Hotels-g189158-oa90-Lisbon_Lisbon_District_Central_Portugal-Hotels.html


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tripadvisor.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Parsing results 
0                         Corinthia Lisbon
1                     Hotel Avenida Palace
2      Corpo Santo Lisbon Historical Hotel
3                        Hotel Santa Justa
4                       H10 Duque de Loule
                      ...                 
119                     Hotel Zenit Lisboa
120                     Flores Guest House
121            Lisbon Old Town Guest House
122                     Casa De Sao Mamede
123        Dinya Lisbon Hotel & Lounge Bar
Name: hotel_name, Length: 124, dtype: object


In [33]:
hlist

['http://www.tripadvisor.com/Hotels-g189158-Lisbon_Lisbon_District_Central_Portugal-Hotels.html',
 'http://www.tripadvisor.com/Hotels-g189158-oa30-Lisbon_Lisbon_District_Central_Portugal-Hotels.html',
 'http://www.tripadvisor.com/Hotels-g189158-oa60-Lisbon_Lisbon_District_Central_Portugal-Hotels.html',
 'http://www.tripadvisor.com/Hotels-g189158-oa90-Lisbon_Lisbon_District_Central_Portugal-Hotels.html']

In [25]:
len(data)

126

In [26]:
my_df = pd.DataFrame(data)

In [34]:
my_df.head(100)

,hotel_name,url,locality,reviews,rank,tripadvisor_rating,checkOut,checkIn,hotel_features,price_per_night,no_of_deals,booking_provider,raw_rank
0,Corinthia Lisbon,http://www.tripadvisor.com/Hotel_Review-g18915...,Lisbon,6772,None,4.5,2021/03/10,2021/03/07,,None,20,Expedia.com,None
1,Hotel Portugal,http://www.tripadvisor.com/Hotel_Review-g18915...,Lisbon,1530,"#1 Best Value of 1,977 places to stay in Lisbon",4.5,2021/03/10,2021/03/07,,$107,20,Booking.com,None
2,"As Janelas Verdes Inn, a Lisbon Heritage Colle...",http://www.tripadvisor.com/Hotel_Review-g18915...,Lisbon,1355,"#2 Best Value of 1,977 places to stay in Lisbon",None,2021/03/10,2021/03/07,,$124,18,Booking.com,None
3,9 Hotel Mercy,http://www.tripadvisor.com/Hotel_Review-g18915...,Lisbon,1378,"#3 Best Value of 1,977 places to stay in Lisbon",4.5,2021/03/10,2021/03/07,,$83,19,Tripadvisor,None
4,PortoBay Liberdade,http://www.tripadvisor.com/Hotel_Review-g18915...,Lisbon,1450,"#4 Best Value of 1,977 places to stay in Lisbon",4.5,2021/03/10,2021/03/07,,$158,19,Booking.com,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Corinthia Lisbon,http://www.tripadvisor.com/Hotel_Review-g18915...,Lisbon,6772,None,4.5,2021/03/10,2021/03/07,,$171,20,Booking.com,None
96,OnJ S. Lazaro,http://www.tripadvisor.com/Hotel_Review-g18915...,Lisbon,163,"#91 Best Value of 1,977 places to stay in Lisbon",4.5,2021/03/10,2021/03/07,,$115,15,Booking.com,None
97,Lisboa Central Hostel,http://www.tripadvisor.com/Hotel_Review-g18915...,Lisbon,856,"#92 Best Value of 1,977 places to stay in Lisbon",None,2021/03/10,2021/03/07,,$62,11,Booking.com,None
98,NH Collection Lisboa Liberdade,http://www.tripadvisor.com/Hotel_Review-g18915...,Lisbon,878,"#93 Best Value of 1,977 places to stay in Lisbon",4.5,2021/03/10,2021/03/07,,$122,20,Booking.com,None


In [8]:
print(my_df['hotel_name'])

0                                       Corinthia Lisbon
1      As Janelas Verdes Inn, a Lisbon Heritage Colle...
2                                          9 Hotel Mercy
3                          Hotel Ibis Lisboa Jose Malhoa
4                                     Hotel White Lisboa
                             ...                        
121                                   Hotel Zenit Lisboa
122                                   Flores Guest House
123                          Lisbon Old Town Guest House
124                                   Casa De Sao Mamede
125                      Dinya Lisbon Hotel & Lounge Bar
Name: hotel_name, Length: 126, dtype: object


In [9]:
for i in data:
    for y in i:
        print (y,':',i[y])

hotel_name : Corinthia Lisbon
url : http://www.tripadvisor.com/Hotel_Review-g189158-d276952-Reviews-Corinthia_Lisbon-Lisbon_Lisbon_District_Central_Portugal.html
locality : Lisbon
reviews : 6773 
rank : None
tripadvisor_rating : 4.5
checkOut : 2021/03/10
checkIn : 2021/03/07
hotel_features : 
price_per_night : None
no_of_deals : 20
booking_provider : Hotels.com
hotel_name : As Janelas Verdes Inn, a Lisbon Heritage Collection
url : http://www.tripadvisor.com/Hotel_Review-g189158-d195740-Reviews-As_Janelas_Verdes_Inn_a_Lisbon_Heritage_Collection-Lisbon_Lisbon_District_Central_Portu.html
locality : Lisbon
reviews : 1356 
rank : #1 Best Value of 1,977 places to stay in Lisbon
tripadvisor_rating : None
checkOut : 2021/03/10
checkIn : 2021/03/07
hotel_features : 
price_per_night : $126
no_of_deals : 19
booking_provider : Tripadvisor
hotel_name : 9 Hotel Mercy
url : http://www.tripadvisor.com/Hotel_Review-g189158-d3298291-Reviews-9_Hotel_Mercy-Lisbon_Lisbon_District_Central_Portugal.html
loca

getting all the reviews


In [10]:
location_list

['http://www.tripadvisor.com/Hotels-g189158-Lisbon_Lisbon_District_Central_Portugal-Hotels.html',
 'http://www.tripadvisor.com/Hotels-g189158-oa30-Lisbon_Lisbon_District_Central_Portugal-Hotels.html',
 'http://www.tripadvisor.com/Hotels-g189158-oa60-Lisbon_Lisbon_District_Central_Portugal-Hotels.html',
 'http://www.tripadvisor.com/Hotels-g189158-oa90-Lisbon_Lisbon_District_Central_Portugal-Hotels.html']

In [11]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import requests
from bs4 import BeautifulSoup as soup

#http://www.tripadvisor.com/Hotels-g187147-Paris_Ile_de_France-Hotels.html
#location_list = 'http://www.tripadvisor.com/Hotels-g61000-Yosemite_National_Park_California-Hotels.html'
# In[2]:

header = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
html = requests.get(location_list[0],headers=header)
html2 = requests.get(location_list[1],headers=header)
html3 = requests.get(location_list[2],headers=header)
html.status_code

200

In [13]:



# In[3]:


bsobj = soup(html.content,'lxml')
print('first_step')


# In[4]:


links = []

for review in bsobj.findAll('a',{'class':'review_count'}):
    a = review['href']
    a = 'https://www.tripadvisor.com'+ a
    b = review['href']
    b = 'https://www.tripadvisor.com'+ b
    c = review['href']
    c = 'https://www.tripadvisor.com'+ c
    d = review['href']
    d = 'https://www.tripadvisor.com'+ d
    e = review['href']
    e = 'https://www.tripadvisor.com'+ e
    links.append('https://www.tripadvisor.com/'+a)
    print(a)
    #hotel_name = a.find('h1', id='HEADING').text
        
    print(a.find('Reviews'))
    a = a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
    b = b[:(b.find('Reviews')+7)] + '-or5' + b[(b.find('Reviews')+7):]
    c = c[:(c.find('Reviews')+7)] + '-or10' + c[(c.find('Reviews')+7):]
    d = d[:(d.find('Reviews')+7)] + '-or15' + d[(d.find('Reviews')+7):]
    e = e[:(e.find('Reviews')+7)] + '-or20' + e[(e.find('Reviews')+7):]

    #print(hotel_name)
    links.append(a)
    links.append(b)
    links.append(c)
    links.append(d)
    links.append(e)
#links


# In[5]:
bsobj = soup(html2.content,'lxml')
print('second_step')


# In[4]:




for review in bsobj.findAll('a',{'class':'review_count'}):
    a = review['href']
    a = 'https://www.tripadvisor.com'+ a
    b = review['href']
    b = 'https://www.tripadvisor.com'+ b
    c = review['href']
    c = 'https://www.tripadvisor.com'+ c
    d = review['href']
    d = 'https://www.tripadvisor.com'+ d
    e = review['href']
    e = 'https://www.tripadvisor.com'+ e
    links.append('https://www.tripadvisor.com/'+a)
    print(a)
    #hotel_name = a.find('h1', id='HEADING').text
        
    print(a.find('Reviews'))
    a = a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
    b = b[:(b.find('Reviews')+7)] + '-or5' + b[(b.find('Reviews')+7):]
    c = c[:(c.find('Reviews')+7)] + '-or10' + c[(c.find('Reviews')+7):]
    d = d[:(d.find('Reviews')+7)] + '-or15' + d[(d.find('Reviews')+7):]
    e = e[:(e.find('Reviews')+7)] + '-or20' + e[(e.find('Reviews')+7):]

    #print(hotel_name)
    links.append(a)
    links.append(b)
    links.append(c)
    links.append(d)
    links.append(e)

bsobj = soup(html3.content,'lxml')
print('third_step')
for review in bsobj.findAll('a',{'class':'review_count'}):
    a = review['href']
    a = 'https://www.tripadvisor.com'+ a
    b = review['href']
    b = 'https://www.tripadvisor.com'+ b
    c = review['href']
    c = 'https://www.tripadvisor.com'+ c
    d = review['href']
    d = 'https://www.tripadvisor.com'+ d
    e = review['href']
    e = 'https://www.tripadvisor.in'+ e
    links.append('https://www.tripadvisor.com/'+a)
    print(a)
    #hotel_name = a.find('h1', id='HEADING').text
        
    print(a.find('Reviews'))
    a = a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
    b = b[:(b.find('Reviews')+7)] + '-or5' + b[(b.find('Reviews')+7):]
    c = c[:(c.find('Reviews')+7)] + '-or10' + c[(c.find('Reviews')+7):]
    d = d[:(d.find('Reviews')+7)] + '-or15' + d[(d.find('Reviews')+7):]
    e = e[:(e.find('Reviews')+7)] + '-or20' + e[(e.find('Reviews')+7):]

    #print(hotel_name)
    links.append(a)
    links.append(b)
    links.append(c)
    links.append(d)
    links.append(e)



first_step
https://www.tripadvisor.com/Hotel_Review-g189158-d276952-Reviews-Corinthia_Lisbon-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS
57
https://www.tripadvisor.com/Hotel_Review-g189158-d195643-Reviews-Hotel_Avenida_Palace-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS
57
https://www.tripadvisor.com/Hotel_Review-g189158-d12659702-Reviews-Corpo_Santo_Lisbon_Historical_Hotel-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS
59
https://www.tripadvisor.com/Hotel_Review-g189158-d4257306-Reviews-Hotel_Santa_Justa-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS
58
https://www.tripadvisor.com/Hotel_Review-g189158-d7787341-Reviews-H10_Duque_de_Loule-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS
58
https://www.tripadvisor.com/Hotel_Review-g189158-d195742-Reviews-Hotel_Britania_a_Lisbon_Heritage_Collection-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS
57
https://www.tripadvisor.com/Hotel_Review-g189158-d7363512-Reviews-Jupiter_Lisboa_Hotel-Lisbon_Lisbon_Dis

In [14]:
links[0:9]

['https://www.tripadvisor.com/https://www.tripadvisor.com/Hotel_Review-g189158-d276952-Reviews-Corinthia_Lisbon-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS',
 'https://www.tripadvisor.com/Hotel_Review-g189158-d276952-Reviews-or{}-Corinthia_Lisbon-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS',
 'https://www.tripadvisor.com/Hotel_Review-g189158-d276952-Reviews-or5-Corinthia_Lisbon-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS',
 'https://www.tripadvisor.com/Hotel_Review-g189158-d276952-Reviews-or10-Corinthia_Lisbon-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS',
 'https://www.tripadvisor.com/Hotel_Review-g189158-d276952-Reviews-or15-Corinthia_Lisbon-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS',
 'https://www.tripadvisor.com/Hotel_Review-g189158-d276952-Reviews-or20-Corinthia_Lisbon-Lisbon_Lisbon_District_Central_Portugal.html#REVIEWS',
 'https://www.tripadvisor.com/https://www.tripadvisor.com/Hotel_Review-g189158-d195643-Reviews-Hotel_Avenida_Palac

In [15]:

from random import randint
from time import sleep

reviews = []
all = []

for link in links:
    d = [5,10,15,20,25]
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
    html2 = requests.get(link.format(i for i in range(5,1000,5)),headers=headers)
    sleep(randint(1,5))
    bsobj2 = soup(html2.content,'lxml')
    hotel_name = bsobj2.find('h1', id='HEADING').text
    #print(hotel_name)
    
    
    for r in bsobj2.findAll('q'):
        if r.span.text:
            reviews.append(r.span.text.strip())
        #print(link)
        #print(r.span.text.strip())
            all.append([hotel_name,link,r.span.text.strip()])
        
#reviews


AttributeError: 'NoneType' object has no attribute 'text'

In [16]:
combined = pd.DataFrame(all,columns=['Hotel','link','review'])

In [17]:
len(combined['Hotel'].drop_duplicates())

59

In [18]:
combined['Hotel'].value_counts()

Corinthia Lisbon                                            90
My Story Hotel Tejo                                         30
Memmo Principe Real                                         30
The Vintage Lisbon                                          30
LX Boutique Hotel                                           30
Brown's Central Hotel                                       30
We Love F. Tourists - Hostel                                30
Internacional Design Hotel                                  30
PortoBay Marques                                            30
Tivoli Avenida Liberdade Lisboa                             30
Bairro Alto Hotel                                           30
Tesouro da Baixa by Shiadu                                  30
Vincci Liberdade                                            30
EVOLUTION Lisboa Hotel                                      30
Brown's Downtown Hotel                                      30
The One Palacio da Anunciada                           

In [19]:
combined.to_csv("combined_lisbon2.csv",index=False)

In [92]:
len(combined['Hotel'].drop_duplicates())

52

#change to auto run later

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Paris"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Chicago"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "New York"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "San Francisco"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Istanbul"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Singapore"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bombay"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bangkok"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Sydney"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "London"

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Rome"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Athens"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Miami"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Austin"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Tokyo"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bali"

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Osaka"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Hong Kong"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Dubai"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Phuket"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Seoul"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Prague"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Milan"

In [ ]:
test="2020/11/01,2020/11/05,popularity,Milan"

In [ ]:
test.split(',')

In [ ]:
args
print(locality)

In [ ]:
if today<datetime.strptime(checkIn,"%Y/%m/%d") and datetime.strptime(checkIn,"%Y/%m/%d")<datetime.strptime(checkOut,"%Y/%m/%d"):
        data = parse(locality,checkin_date,checkout_date,sort)
        print ("Writing to output file tripadvisor_data.csv")
        with open('tripadvisor_data.csv','wb')as csvfile:
            fieldnames = ['hotel_name','url','locality','reviews','tripadvisor_rating','checkIn','checkOut','price_per_night','booking_provider','no_of_deals','hotel_features']
            my_df  = pd.DataFrame(columns = fieldnames)
            for row in  data:
                my_df.loc[len(my_df)] = [row['hotel_name'],row['url'],row['locality'],row['reviews'],row['tripadvisor_rating'],row['checkIn'],row['checkOut'],row['price_per_night'],row['booking_provider'],row['no_of_deals'],row['hotel_features']]
            DB_COLUMN   = 'review_body'
            DB_COLUMN1 = 'review_date'
            DB_COLUMN2 = 'hotelName'
            DB_COLUMN3 = 'hotelUrl'
            start_urls = []
            for index, row2 in my_df.iterrows():
                start_urls.append(row2['url'])
            lang = 'en'
    
            headers = [
            DB_COLUMN,
            DB_COLUMN1,
            DB_COLUMN2,
            DB_COLUMN3
            ]
            reviews_df  = pd.DataFrame(columns = headers)
            
            for url in start_urls:

                # get all reviews for 'url' and 'lang'
                items = scrape(url, lang)
                filename = 'hotelReviewsIn' + locality + '__' + lang
                filename2 = "HotelListIn" + locality + '__' + lang
                if not items:
                    print('No reviews')
                else:
                    # write in CSV
                    for i in items:
                        reviews_df.loc[len(reviews_df)] = [i['review_body'],i['review_date'],my_df.loc[my_df['url'] == url]['hotel_name'], url]
            ci = checkIn.replace("/", "")
            co = checkOut.replace("/", "")
            reviews_df.to_csv(filename + ci +co+'.csv', encoding='utf-8')     
            my_df.to_csv(filename2 + ci +co+'.csv', encoding='utf-8')       
    #checking whether the entered date is already passed
elif today>datetime.strptime(checkIn,"%Y/%m/%d") or today>datetime.strptime(checkOut,"%Y/%m/%d"):
     print ("Invalid Checkin date: Please enter a valid checkin and checkout dates,entered date is already passed")

elif datetime.strptime(checkIn,"%Y/%m/%d")>datetime.strptime(checkOut,"%Y/%m/%d"):
     print ("Invalid Checkin date: CheckIn date must be less than checkOut date")


In [6]:
data = parse(locality,checkin_date,checkout_date,sort)

NameError: name 'locality' is not defined

In [ ]:
len(data)

In [ ]:
http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html
https://www.tripadvisor.com/Hotels-g187849-oa30-Milan_Lombardy-Hotels.html
https://www.tripadvisor.com/Hotels-g187849-oa60-Milan_Lombardy-Hotels.html

In [ ]:
a="http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html"

In [ ]:
b=a.split("-")

In [ ]:
print (b)

In [ ]:
print(b[0])

In [ ]:
s="-"

In [ ]:
c=s.join([b[0],b[1],"oa30",b[2],b[3]])
d=s.join([b[0],b[1],"oa60",b[2],b[3]])

In [ ]:
urllist = [a,c,d]

In [ ]:
urllist